In [60]:
from keys import client_id, client_secret, user_name
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy import oauth2
import random
from functools import reduce

In [61]:
scope = 'user-library-read playlist-modify-public playlist-modify-private playlist-read-private user-read-recently-played app-remote-control user-top-read'

redirect_uri = 'https://developer.spotify.com/dashboard/applications/0743a195f7654b5ab95560a95e89316a'

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

token = util.prompt_for_user_token(user_name, scope, client_id, client_secret, redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("NO TOKEN FOUND")


In [62]:
user_top_tracks = sp.current_user_top_tracks(limit=50, offset=0, time_range='long_term')
song_data = user_top_tracks["items"]

song_ids = []
song_names = []

for i in range(0, len(song_data)):
    if song_data[i]['id'] != None:
        song_ids.append(song_data[i]['id'])
        song_names.append(song_data[i]['name'])

song_features = []
for i in range(0, len(song_ids)):
    features = sp.audio_features(song_ids[i])
    for song in features:
        song_features.append(song)

top_tracks_df = pd.DataFrame(song_features, index=song_names)
top_tracks_df = top_tracks_df[["id", "acousticness", "danceability", "duration_ms", 
                         "energy", "instrumentalness",  "key", "liveness",
                         "loudness", "mode", "speechiness", "tempo", "valence"]]
top_tracks_df.head()

,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
HOLIDAY,6zFMeegAMYQo0mt8rXtrli,0.12000,0.810,154998,0.511,0.000000,5,0.0832,-6.924,0,0.1640,151.947,0.837
Death of a Bachelor,1BECwm5qkaBwlbfo4kpYx8,0.01370,0.462,203507,0.538,0.000000,0,0.4290,-5.527,1,0.0590,139.256,0.405
Daylight,6Ed1q0X8oSKSm4IIhiQbYg,0.00488,0.528,163906,0.749,0.000096,7,0.0949,-7.571,1,0.0479,163.944,0.729
Knock Knock,3uYm4MtU6jUQft2DtGqEoZ,0.03530,0.667,202807,0.922,0.000000,4,0.3640,-4.857,0,0.1280,108.952,0.509
Don't Matter,7I6DceMT3utDOHjcYCbrr4,0.24400,0.798,293053,0.443,0.000000,2,0.3260,-6.062,1,0.0403,125.232,0.354


In [63]:
featured_playlists = sp.featured_playlists(limit=50)
# Creating data frame for playlists
id = []
name = []
num_tracks = []
items = featured_playlists['playlists']['items']
for item in items:
    id.append(item["id"])
    name.append(item["name"])
    num_tracks.append(item["tracks"]["total"])

df_pl_data = pd.DataFrame({"id":id, "name": name, "num_tracks": num_tracks})

In [64]:
# Gathering playlists Categories
categories = sp.categories(limit=50)['categories']['items']
category_id = []
category_name = []
for item in categories:
    category_id.append(item['id'])
    category_name.append(item['name'])
categories_2 = sp.categories(limit=50, offset=50)['categories']['items']
for item in categories_2:
    category_id.append(item['id'])
    category_name.append(item['name'])

In [65]:
# Creating category dataframe
data_category = {'Name': category_name,
                 'id': category_id}
category_df = pd.DataFrame(data_category)

category_df

,Name,id
0,Top Lists,toplists
1,Hip Hop,hiphop
2,Pop,pop
3,Country,country
4,Rock,rock
5,Mood,mood
6,Workout,workout
7,Latin,latin
8,R&B,rnb
9,Stop Asian Hate,aapi_heritage_month


In [95]:
# Extracting playlists from categories
sp.category_playlists('toplists')['playlists']['items'][0]
category_pl_id = []
category_pl_name = []
category_pl_num_tracks = []
for pl_id in category_id:
    PL = sp.category_playlists(pl_id)['playlists']['items']
    for i in range(len(PL)):
        category_pl_id.append(PL[i]['id'])
        category_pl_name.append(PL[i]['name'])
        category_pl_num_tracks.append(PL[i]['tracks']['total'])

976

In [67]:
# Converting category playlist data into a DataFrame
category_pl_data = {'id': category_pl_id,
                    'name': category_pl_name,
                    'num_tracks': category_pl_num_tracks}

category_pl_df = pd.DataFrame(category_pl_data)

In [96]:
category_pl_df['num_tracks'].sum()

84187

In [69]:
df_pl_data

,id,name,num_tracks
0,37i9dQZF1DXbYM3nMM0oPk,Mega Hit Mix,75
1,37i9dQZF1DXatMjChPKgBk,Dinner Music,150
2,37i9dQZF1DWYgsiQBNyGuA,Soulful Day,80
3,37i9dQZF1DWYWddJiPzbvb,Comfort Zone,150
4,37i9dQZF1DWZd79rJ6a7lp,Sleep,186
5,37i9dQZF1DWZ0OzPeadl0h,Lush + Ethereal,99
6,37i9dQZF1DWStLt4f1zJ6I,Songs For Sleeping,99
7,37i9dQZF1DX0QKpU3cGsyb,Bedroom Jams,70
8,37i9dQZF1DWU0ScTcjJBdj,Relax & Unwind,100
9,37i9dQZF1DWTwnEm1IYyoj,Soft Pop Hits,241


In [70]:
# Returns Individual Tracks In Playlists
def get_playlist_tracks(sp, playlist_id):
    data_track = sp.playlist_tracks(playlist_id, fields=None, limit=100, offset=0, market=None)['items']
    track_id = []
    track_name = []
    
    for data in data_track:
        track_id.append(data['track']['id'])
        track_name.append(data['track']['name'])

    return pd.DataFrame({"track_id":track_id, "track_name": track_name})

In [71]:
get_playlist_tracks(sp, '37i9dQZF1DWZKuerrwoAGz').head()

,track_id,track_name
0,6FE2iI43OZnszFLuLtvvmg,Classic
1,3DmW6y7wTEYHJZlLo1r6XJ,Shower
2,3E7dfMvvCLUddWissuqMwr,Party In The U.S.A.
3,4E5P1XyAFtrjpiIxkydly4,Replay
4,1CQ2cMfrmFM1YdfmjENKVE,She Looks So Perfect


In [72]:
# Function for getting audio features of individual tracks
def get_audio_features(sp, playlist_id):
    playlist = get_playlist_tracks(sp, playlist_id)
    audio_features = []
    for i in range(len(playlist)):
        song_data_id = playlist['track_id'][i]
        audio_features.append(sp.audio_features(song_data_id))
    features_array = []
    for features in audio_features:
        features = features[0]
        features_array.append([features['danceability'],
                              features['acousticness'],
                              features['energy'], 
                              features['tempo'],
                              features['instrumentalness'], 
                              features['loudness'],
                              features['liveness'],
                              features['duration_ms'],
                              features['key'],
                              features['valence'],
                              features['speechiness']
                             ])
    df_audio_features = pd.DataFrame(features_array, columns=['danceability', 'acousticness', 'energy', 'tempo', 'instrumentalness', 'loudness', 'liveness', 'duration_ms', 'key', 'valence', 'speechiness'])
    # DataFrame with audio features and playlist id/name
    df_playlist_features = pd.concat([playlist, df_audio_features], axis=1)
    df_playlist_features.set_index('track_name', inplace=True, drop=True)
    return df_playlist_features

In [73]:
get_audio_features(sp, '37i9dQZF1DXcRXFNfZr7Tp').head()

,track_id,danceability,acousticness,energy,tempo,instrumentalness,loudness,liveness,duration_ms,key,valence,speechiness
track_name,,,,,,,,,,,,
Beautiful Mistakes (feat. Megan Thee Stallion),6fRxMU4LWwyaSSowV441IU,0.713,0.0377,0.676,99.048,0.000000,-5.483,0.154,227395,10,0.721,0.0270
Met Him Last Night (feat. Ariana Grande),0BI0hfbmqybnd3TezrDME3,0.538,0.2200,0.512,144.978,0.000000,-4.548,0.101,204632,4,0.120,0.0262
BED,7jXQUrVhEpXdymfFWNDnQW,0.663,0.0134,0.783,123.986,0.001790,-4.585,0.325,178088,6,0.622,0.0393
Peaches (feat. Daniel Caesar & Giveon),4iJyoBOLtHqaGxP12qzhQI,0.677,0.3210,0.696,90.030,0.000000,-6.181,0.420,198082,0,0.464,0.1190
deja vu,61KpQadow081I2AsbeLcsb,0.439,0.5930,0.610,181.088,0.000011,-7.236,0.341,215508,9,0.172,0.1160


In [74]:
# Finding mean of audio features for each playlist
def audio_features_mean(sp, playlist_id):
    Playlist = get_audio_features(sp, playlist_id)
    return pd.DataFrame(Playlist.mean(), columns= [playlist_id])

In [75]:
audio_features_mean(sp, '37i9dQZF1DXbYM3nMM0oPk')

,37i9dQZF1DXbYM3nMM0oPk
danceability,0.709640
acousticness,0.178981
energy,0.647813
tempo,120.666040
instrumentalness,0.008807
loudness,-5.957933
liveness,0.165035
duration_ms,188330.800000
key,5.333333
valence,0.546085


In [76]:
# Merges average of audio features of each playlist into a single DataFrame
primary_df = []
for i in range(len(df_pl_data)):
    primary_df.append(audio_features_mean(sp, df_pl_data['id'][i]))

In [77]:
primary_df

[                  37i9dQZF1DXbYM3nMM0oPk
 danceability                    0.709640
 acousticness                    0.178981
 energy                          0.647813
 tempo                         120.666040
 instrumentalness                0.008807
 loudness                       -5.957933
 liveness                        0.165035
 duration_ms                188330.800000
 key                             5.333333
 valence                         0.546085
 speechiness                     0.119236,
                   37i9dQZF1DXatMjChPKgBk
 danceability                    0.598160
 acousticness                    0.562616
 energy                          0.392099
 tempo                         110.214330
 instrumentalness                0.011643
 loudness                       -9.402780
 liveness                        0.142804
 duration_ms                209992.550000
 key                             4.640000
 valence                         0.400028
 speechiness                     

In [78]:
# Turns playlist feature mean data into correctly formatted DataFrame
X_data = reduce(lambda left,right: pd.merge(left,right, left_index=True, right_index=True), primary_df)
X_data

,37i9dQZF1DXbYM3nMM0oPk,37i9dQZF1DXatMjChPKgBk,37i9dQZF1DWYgsiQBNyGuA,37i9dQZF1DWYWddJiPzbvb,37i9dQZF1DWZd79rJ6a7lp,37i9dQZF1DWZ0OzPeadl0h,37i9dQZF1DWStLt4f1zJ6I,37i9dQZF1DX0QKpU3cGsyb,37i9dQZF1DWU0ScTcjJBdj,37i9dQZF1DWTwnEm1IYyoj,37i9dQZF1DXcxvFzl58uP7,37i9dQZF1DX79Y9Kr2M2tM
danceability,0.709640,0.598160,0.684975,0.648130,0.160678,0.501434,0.419300,0.624714,0.521320,0.606130,0.677370,0.474828
acousticness,0.178981,0.562616,0.229380,0.324553,0.944250,0.722329,0.953475,0.330599,0.695853,0.358673,0.387600,0.731772
energy,0.647813,0.392099,0.597813,0.556375,0.032744,0.334188,0.078623,0.515786,0.363995,0.554328,0.527340,0.328660
tempo,120.666040,110.214330,112.002063,116.989920,81.433500,120.603465,101.145232,117.082914,117.174500,113.626660,118.548050,113.674929
instrumentalness,0.008807,0.011643,0.065591,0.013410,0.883610,0.166434,0.922566,0.003760,0.074661,0.009761,0.060106,0.219547
loudness,-5.957933,-9.402780,-7.559575,-6.694580,-31.683130,-12.259606,-23.431899,-8.000343,-11.082280,-6.714250,-8.754250,-12.737990
liveness,0.165035,0.142804,0.157073,0.134165,0.106823,0.134213,0.112460,0.164307,0.155497,0.133532,0.179815,0.129309
duration_ms,188330.800000,209992.550000,248808.825000,210468.610000,185802.540000,235180.070707,204477.262626,204737.700000,241161.370000,224676.710000,177599.110000,227599.242424
key,5.333333,4.640000,5.962500,4.930000,5.060000,4.979798,4.313131,5.285714,5.810000,5.490000,4.810000,4.686869
valence,0.546085,0.400028,0.666775,0.419360,0.077998,0.211603,0.195108,0.424427,0.297808,0.416086,0.545914,0.206508


In [79]:
# Turns my favorite track data (Y_data) into a DataFrame with audio features averaged
# maybe change mean to median
instrumental_mean = top_tracks_df['instrumentalness'].mean()
Y_data = pd.DataFrame(top_tracks_df.median(), columns=['top_tracks']).drop('mode')
Y_data.loc['instrumentalness'] = instrumental_mean
Y_data

,top_tracks
acousticness,0.038250
danceability,0.655500
duration_ms,204280.000000
energy,0.701500
instrumentalness,0.000695
key,7.000000
liveness,0.127500
loudness,-5.165500
speechiness,0.060100
tempo,120.641500


In [80]:
# Model for predicting playlist of "best-fit" using feature_importances_
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=44, max_depth=5, max_features=8, oob_score=True)
model.fit(X_data, Y_data['top_tracks'])
pl_rank = model.feature_importances_
pl_importances = pd.DataFrame(pl_rank, index = X_data.columns, columns=['importance']).sort_values('importance',                                        ascending=False)

pl_importances

,importance
37i9dQZF1DWZd79rJ6a7lp,0.323325
37i9dQZF1DWStLt4f1zJ6I,0.130319
37i9dQZF1DWYgsiQBNyGuA,0.106142
37i9dQZF1DXatMjChPKgBk,0.075573
37i9dQZF1DWYWddJiPzbvb,0.071162
37i9dQZF1DX79Y9Kr2M2tM,0.064198
37i9dQZF1DX0QKpU3cGsyb,0.048261
37i9dQZF1DXbYM3nMM0oPk,0.045988
37i9dQZF1DWTwnEm1IYyoj,0.043980
37i9dQZF1DWZ0OzPeadl0h,0.042593


In [81]:
pl_t3 = pl_importances.index[0:3]
frames = []
for i in range(len(pl_t3)):
    frame = get_audio_features(sp, pl_t3[i])
    frames.append(frame)
    
recommended_pl = pd.concat(frames)

In [82]:
recommended_pl.head()

,track_id,danceability,acousticness,energy,tempo,instrumentalness,loudness,liveness,duration_ms,key,valence,speechiness
track_name,,,,,,,,,,,,
Viento Estelar,0f5IlhiVWUUq4SGHyrRxIt,0.141,0.965,0.04050,59.418,0.966,-29.013,0.0983,185943,3,0.0378,0.0382
Beautiful Imagination,5LEHlRPjGcZ5RdagAwXpHS,0.102,0.797,0.03330,89.605,0.927,-34.177,0.1110,143053,3,0.0591,0.0476
Nepenthe,0kKOC50fPTnqmrYvtJInsT,0.133,0.993,0.00512,67.818,0.851,-33.295,0.0954,151750,9,0.0495,0.0380
Lenitivo,6PEwEkL4lzRbGMGH7j7kBB,0.183,0.987,0.01230,72.183,0.785,-34.571,0.0869,180625,9,0.0636,0.0409
Campus,2PUoynqedBx15JbPYxzA5v,0.145,0.878,0.01150,72.120,0.855,-28.292,0.1220,219474,0,0.0320,0.0421


In [83]:
pl_rec = recommended_pl.set_index('track_id')
pl_rec.sort_index( axis=1, level=None, ascending=True, inplace=True, kind='quicksort')

pl_rec.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,valence
track_id,,,,,,,,,,,
0f5IlhiVWUUq4SGHyrRxIt,0.965,0.141,185943,0.04050,0.966,3,0.0983,-29.013,0.0382,59.418,0.0378
5LEHlRPjGcZ5RdagAwXpHS,0.797,0.102,143053,0.03330,0.927,3,0.1110,-34.177,0.0476,89.605,0.0591
0kKOC50fPTnqmrYvtJInsT,0.993,0.133,151750,0.00512,0.851,9,0.0954,-33.295,0.0380,67.818,0.0495
6PEwEkL4lzRbGMGH7j7kBB,0.987,0.183,180625,0.01230,0.785,9,0.0869,-34.571,0.0409,72.183,0.0636
2PUoynqedBx15JbPYxzA5v,0.878,0.145,219474,0.01150,0.855,0,0.1220,-28.292,0.0421,72.120,0.0320


In [84]:
Y_data_sorted = Y_data.sort_index( axis=0, level=None, ascending=True, inplace=False, kind='quicksort').squeeze(1)

Y_data_sorted.shape

(11,)

In [85]:
Y_data_sorted

acousticness             0.038250
danceability             0.655500
duration_ms         204280.000000
energy                   0.701500
instrumentalness         0.000695
key                      7.000000
liveness                 0.127500
loudness                -5.165500
speechiness              0.060100
tempo                  120.641500
valence                  0.542000
Name: top_tracks, dtype: float64

In [86]:
df_main_var = pl_rec.subtract(Y_data_sorted, axis=1)
df_main_var = df_main_var.divide(Y_data_sorted, axis=1) 

df_main_var.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,valence
track_id,,,,,,,,,,,
0f5IlhiVWUUq4SGHyrRxIt,24.228758,-0.784897,-0.089764,-0.942267,1389.262521,-0.571429,-0.229020,4.616688,-0.364393,-0.507483,-0.930258
5LEHlRPjGcZ5RdagAwXpHS,19.836601,-0.844394,-0.299721,-0.952530,1333.133910,-0.571429,-0.129412,5.616397,-0.207987,-0.257262,-0.890959
0kKOC50fPTnqmrYvtJInsT,24.960784,-0.797101,-0.257147,-0.992701,1223.755078,0.285714,-0.251765,5.445649,-0.367720,-0.437855,-0.908672
6PEwEkL4lzRbGMGH7j7kBB,24.803922,-0.720824,-0.115797,-0.982466,1128.768197,0.285714,-0.318431,5.692673,-0.319468,-0.401674,-0.882657
2PUoynqedBx15JbPYxzA5v,21.954248,-0.778795,0.074378,-0.983607,1229.511858,-1.000000,-0.043137,4.477108,-0.299501,-0.402196,-0.940959


In [87]:
df_main_var['variation'] = df_main_var.sum(axis=1)

df_main_var['variation'] = df_main_var['variation'].abs()

In [88]:
df_main_var

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,valence,variation
track_id,,,,,,,,,,,,
0f5IlhiVWUUq4SGHyrRxIt,24.228758,-0.784897,-0.089764,-0.942267,1389.262521,-0.571429,-0.229020,4.616688,-0.364393,-0.507483,-0.930258,1413.688457
5LEHlRPjGcZ5RdagAwXpHS,19.836601,-0.844394,-0.299721,-0.952530,1333.133910,-0.571429,-0.129412,5.616397,-0.207987,-0.257262,-0.890959,1354.433215
0kKOC50fPTnqmrYvtJInsT,24.960784,-0.797101,-0.257147,-0.992701,1223.755078,0.285714,-0.251765,5.445649,-0.367720,-0.437855,-0.908672,1250.434264
6PEwEkL4lzRbGMGH7j7kBB,24.803922,-0.720824,-0.115797,-0.982466,1128.768197,0.285714,-0.318431,5.692673,-0.319468,-0.401674,-0.882657,1155.809190
2PUoynqedBx15JbPYxzA5v,21.954248,-0.778795,0.074378,-0.983607,1229.511858,-1.000000,-0.043137,4.477108,-0.299501,-0.402196,-0.940959,1251.569398
...,...,...,...,...,...,...,...,...,...,...,...,...
7a5xvCgoD8qFEQmO8Jjvtf,0.866667,0.090770,-0.008753,-0.144690,1.374672,-0.285714,-0.312941,0.496080,-0.560732,-0.278946,0.420664,1.657077
69YW4FrGpsjzt9lr517bUz,14.843137,-0.310450,0.102080,-0.205987,-1.000000,0.285714,1.219608,0.335592,-0.415973,-0.076918,0.254613,15.031416
1emxi7e5EWFsJmwSKTuUDH,2.241830,-0.257056,0.207034,-0.291518,239.345591,-0.428571,-0.145098,0.118188,0.064892,0.328871,-0.640221,240.543942


In [89]:
# Picking 30 least variated songs from df_main_var
rec_songs = df_main_var.nsmallest(30,'variation')

rec_songs

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,valence,variation
track_id,,,,,,,,,,,,
6GXbpne1EKdVlYhFazWFs4,0.539869,0.231121,-0.153794,-0.101924,-0.815783,0.285714,-0.184314,0.626367,-0.234609,0.060738,-0.254613,0.001227
1HwgiZluTgktb8SOFqcKkh,0.430065,-0.005339,0.478167,-0.016393,-0.972655,0.285714,-0.496471,0.026038,-0.158070,-0.237020,0.618081,0.047883
5PMKzsUsTpZZGsCcJBuhP2,0.050980,0.302822,0.193132,-0.225944,-0.917534,0.142857,-0.671373,0.193689,0.404326,-0.145966,0.601476,0.071534
1tdarjaWSw6cxpFdLNANuQ,-0.205229,-0.116705,0.134913,0.036351,-0.971360,0.428571,0.349020,-0.056819,-0.447587,0.414049,0.632841,0.198045
6IvKGL3DMPRPjJi8pz059W,-0.301961,-0.411137,0.677829,0.194583,-0.909762,-0.714286,1.133333,0.433550,0.021631,0.417108,-0.195572,0.345316
57Sue0eOgweOlVxc6g2yCo,1.640523,-0.443173,0.483586,-0.218817,-1.000000,0.428571,-0.683137,0.808150,-0.339434,-0.154901,-0.142066,0.379302
3zYQ1KXzqBWRsxIqdVMoYF,0.283660,0.087719,-0.132563,0.140413,-0.885440,0.000000,-0.277647,0.149550,-0.419301,0.159982,0.501845,0.391781
1uyj6ycPIp7i907LsGbUmA,0.484967,0.022121,0.320281,-0.099073,-0.752458,0.000000,-0.699608,0.538476,0.896839,-0.252173,0.483395,0.942766
6h7oewjyzSKywtot8sHXj7,-0.667974,0.031274,0.323218,-0.294369,0.427682,0.285714,-0.378824,1.033491,-0.500832,0.039725,0.678967,0.978073


In [90]:
# Creates custom playlist
def custom_playlist(sp, username, playlist_name, playlist_description, isPublic):
    pl_custom = sp.user_playlist_create(username, playlist_name, public=isPublic, collaborative=False, description=playlist_description)

custom_playlist(sp, user_name, 'DJ_NumPy', 'Custom Playlist', False)

In [91]:
custom_id = sp.user_playlists(user_name)['items'][0]['id']

In [92]:
# Fills custom playlist with recommended tracks
def fill_playlist(playlist_id):
    # try:
    #     sp.user_playlist_unfollow(user=user_name, playlist_id=custom_id)
    # except:
    #     sp.playlist_add_items(playlist_id, items= rec_songs.index[0:len(rec_songs.index)])
    sp.playlist_add_items(playlist_id, items=rec_songs.index[0:len(rec_songs.index)])

In [93]:
fill_playlist(custom_id)